## Import libraries and dependencies

In [3]:
import pandas as pd

import datetime

# Import quantstats
import quantstats as qs
qs.extend_pandas()
# Import yfinance
import yfinance as yf

# Visualization tools
import plotly.express as px

import panel as pn
pn.extension("plotly")

import hvplot.pandas


# Dependent scripts
import fetch_data as fd

## Function to request user input for the cryptocurrencies they have, and the number of years back for which data is to be fetched.

In [8]:
def input_cryptos():
    '''
    Requests user to key in crypto tickers, and appends to a list
    '''
    ticker_list = []
    ticker = ""
    print("Please enter the tickers of your cryptocurrencies one by one")
    print("Type 'done' when finished.")
    while ticker.casefold() != "done":
        ticker = str.upper(input("Ticker: "))
       
                           
        if ticker.casefold() != "done":
            ticker_list.append(ticker)
        else:
            break
    print(f"--------------------------")                      
    print(f"Crpytocurrencies selected:")
    print(*ticker_list, sep=', ')
    print(f"--------------------------")  
    
    # Adds "-USD" suffix to each crypto ticker for compatibility with yfinance
    ticker_list = pd.DataFrame(columns=ticker_list).add_suffix('-USD').columns.tolist()
    return ticker_list

        
ticker_list = input_cryptos() 
                        

Please enter the tickers of your cryptocurrencies one by one
Type 'done' when finished.


Ticker:  eth
Ticker:  btc
Ticker:  luna
Ticker:  done


--------------------------
Crpytocurrencies selected:
ETH, BTC, LUNA
--------------------------


In [9]:
# Function to get ticker data. years_back set to 3 by default, given most altcoins don't have a price history dating back more than that. 
def get_ticker_data(ticker_list, years_back = 3):
    ''''
    Fetches OHLCV data from Yahoo Finance API

    Parameters:
    ticker_list (list): A list of cryptocurrency tickers for which data is to be fetched
    years_back (int): Number of years back from the current date, for which data is to be fetched
    '''
    # Initialise dict to record OHLCV data
    d ={}
    # Define start and end dates
    end = datetime.datetime.now()
    start = datetime.datetime.now() - datetime.timedelta(days= 365 * years_back)
    # create empty dataframe
    cryptos_final = pd.DataFrame()
    
    # Get timeframe 
    interval_list = ['1d', '5d', '1wk', '1mo', '3mo']
    interval = str(input(
        f"\nPlease select the interval: \n"
        f"(Valid intervals: 1d, 5d, 1wk, 1mo, 3mo)\n"
    )
                  )
                  
    while interval not in interval_list:
        print("\nError: You have selected an invalid interval")
        interval = str(input(
            f"Please select the interval: \n"
            f"(Valid intervals: 1d, 5d, 1wk, 1mo, 3mo)"
        )
                      )
                              
    for ticker in ticker_list:
        try:
            # download the crypto price 
            crypto = yf.Ticker(ticker)
            crypto_df = crypto.history(start = start, end = end, interval = interval)
            crypto_df.dropna(inplace = True)
            
            # append the individual crpyto prices 
            if len(crypto_df) == 0:
                None
            else:
                d[(ticker, "open")] = crypto_df['Open']
                d[(ticker, "high")] = crypto_df['High']
                d[(ticker, "low")] = crypto_df["Low"]
                d[(ticker, "close")] = crypto_df["Close"]
                d[(ticker, "volume")] = crypto_df["Volume"]
        except Exception:
            None
    d = pd.DataFrame(d)
    return d


d = get_ticker_data(ticker_list)
d



Please select the interval: 
(Valid intervals: 1d, 5d, 1wk, 1mo, 3mo)
 2d



Error: You have selected an invalid interval


Please select the interval: 
(Valid intervals: 1d, 5d, 1wk, 1mo, 3mo) 1d


ETH-USD                                                      \
                   open         high          low        close       volume   
Date                                                                          
2019-01-08   151.967545   153.625778   148.669540   150.359634   2459808140   
2019-01-09   150.554688   153.622253   150.288376   150.803116   2369241197   
2019-01-10   150.843506   152.148270   126.529373   128.625183   3397734456   
2019-01-11   127.813965   130.165939   125.244942   127.548325   2667585234   
2019-01-12   127.528084   128.666122   125.446754   125.966530   2212109224   
...                 ...          ...          ...          ...          ...   
2022-01-02  3769.298340  3836.161865  3727.357422  3829.564941   9881471548   
2022-01-03  3829.535645  3836.198730  3698.047607  3761.380371  12080777893   
2022-01-04  3761.361572  3876.785156  3723.349854  3794.056641  14030925983   
2022-01-05  3794.269043  3842.059814  3456.745361  3550.386963  18061338502   
2022-01-07  3420.461914  3420.461914  3381.788086  3386.962646  24007157760   

                 BTC-USD                                            \
                    open          high           low         close   
Date                                                                 
2019-01-08   4028.472168   4109.020996   3996.955322   4030.847900   
2019-01-09   4031.552002   4068.403564   4022.662842   4035.296387   
2019-01-10   4034.411377   4064.066650   3659.174561   3678.924561   
2019-01-11   3674.015381   3713.881836   3653.069824   3687.365479   
2019-01-12   3686.973145   3698.978271   3653.810791   3661.301025   
...                  ...           ...           ...           ...   
2022-01-02  47680.925781  47881.406250  46856.937500  47345.218750   
2022-01-03  47343.542969  47510.726562  45835.964844  46458.117188   
2022-01-04  46458.851562  47406.546875  45752.464844  45897.574219   
2022-01-05  45899.359375  46929.046875  42798.222656  43569.003906   
2022-01-07  43152.972656  43152.972656  42867.343750  42957.832031   

                         LUNA-USD                                       
                 volume      open      high       low     close volume  
Date                                                                    
2019-01-08   5306593305  0.001446  0.001572  0.001085  0.001173   11.0  
2019-01-09   5115905225  0.001176  0.001251  0.000925  0.000927    3.0  
2019-01-10   6874143796  0.000928  0.000934  0.000838  0.000844   27.0  
2019-01-11   5538712865  0.000844  0.001062  0.000796  0.000804    8.0  
2019-01-12   4778170883  0.000803  0.000822  0.000782  0.000819   13.0  
...                 ...       ...       ...       ...       ...    ...  
2022-01-02  27951569547  0.009059  0.009060  0.007966  0.008996   22.0  
2022-01-03  33071628362  0.008995  0.009027  0.008709  0.008827   31.0  
2022-01-04  42494677905  0.008827  0.009007  0.008693  0.008721   91.0  
2022-01-05  36851084859  0.008721  0.009345  0.008560  0.008714   56.0  
2022-01-07  29582374912       NaN       NaN       NaN       NaN    NaN  

[1095 rows x 15 columns]